# Laboratorium Big Data
## Systemy rekomendacji - zadanie 2
<i> Antoni Perużyński </i>

In [1]:
import pandas as pd
from sklearn.preprocessing  import StandardScaler

In [5]:
import pyspark
from pyspark.sql import SparkSession

import findspark
findspark.init()
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

Read files

In [6]:
user_artists = spark.read.csv("user_artists.dat",sep='\t', header=True)
artists = spark.read.csv("artists.dat",sep='\t', header=True)
user_artists = user_artists.withColumn("weight", user_artists.weight.cast('int'))
user_artists = user_artists.withColumn("userID", user_artists.userID.cast('int'))
user_artists = user_artists.withColumn("artistID", user_artists.artistID.cast('int'))

In [7]:
user_artists.show()

+------+--------+------+
|userID|artistID|weight|
+------+--------+------+
|     2|      51| 13883|
|     2|      52| 11690|
|     2|      53| 11351|
|     2|      54| 10300|
|     2|      55|  8983|
|     2|      56|  6152|
|     2|      57|  5955|
|     2|      58|  4616|
|     2|      59|  4337|
|     2|      60|  4147|
|     2|      61|  3923|
|     2|      62|  3782|
|     2|      63|  3735|
|     2|      64|  3644|
|     2|      65|  3579|
|     2|      66|  3312|
|     2|      67|  3301|
|     2|      68|  2927|
|     2|      69|  2720|
|     2|      70|  2686|
+------+--------+------+
only showing top 20 rows



Creating extra column "play" which let us to predict and build recommendation. We will use recommendation using 0/1 values - whether a person has listened to a particular performer.

In [25]:
user_artists_Pandas = user_artists.toPandas()
user_artists_Pandas["play"] = 1
user_artists_Pandas

,userID,artistID,weight,play
0,2,51,13883,1
1,2,52,11690,1
2,2,53,11351,1
3,2,54,10300,1
4,2,55,8983,1
...,...,...,...,...
92829,2100,18726,337,1
92830,2100,18727,297,1
92831,2100,18728,281,1
92832,2100,18729,280,1


In [28]:
count_artistID=user_artists_Pandas.groupby(['artistID']).weight.agg('count').to_frame('counartistID').reset_index()

user_artists_Pandas_with_count_artist=user_artists_Pandas.join(count_artistID.set_index("artistID"), on="artistID")

In [30]:
user_artists_Pandas=user_artists_Pandas_with_count_artist

Here is a cell with the ability to drop artists who have been featured in the dataset less than 5 times. E.g., a child may have sent his song, and his parents and family are the only people who have listened to it. Of course, it is also possible to drop new artists whose music has not yet reached the public.

In [12]:
# user_artists_Pandas_with_count_artist_more5=user_artists_Pandas_with_count_artist[user_artists_Pandas_with_count_artist["counartistID"]>10]
# user_artists_Pandas=user_artists_Pandas_with_count_artist_more5

Crearing spark dataframe from pandas dataframe to use ALS from pyspark which give us some recommendation.

In [32]:
sparkDf = spark.createDataFrame(user_artists_Pandas)

C:\Users\DELL\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\Users\DELL\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [33]:
sparkDf.show()

+------+--------+------+----+------------+
|userID|artistID|weight|play|counartistID|
+------+--------+------+----+------------+
|     2|      51| 13883|   1|         111|
|     2|      52| 11690|   1|          23|
|     2|      53| 11351|   1|          75|
|     2|      54| 10300|   1|          18|
|     2|      55|  8983|   1|         298|
|     2|      56|  6152|   1|         151|
|     2|      57|  5955|   1|          19|
|     2|      58|  4616|   1|          82|
|     2|      59|  4337|   1|         107|
|     2|      60|  4147|   1|           2|
|     2|      61|  3923|   1|          25|
|     2|      62|  3782|   1|           3|
|     2|      63|  3735|   1|          41|
|     2|      64|  3644|   1|          71|
|     2|      65|  3579|   1|         369|
|     2|      66|  3312|   1|          19|
|     2|      67|  3301|   1|         429|
|     2|      68|  2927|   1|          17|
|     2|      69|  2720|   1|          29|
|     2|      70|  2686|   1|          85|
+------+---

In [34]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

from pyspark.sql.types import *
from pyspark.sql.functions import col


from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

(ratings_train, ratings_test) = sparkDf.randomSplit([0.7,0.3], seed = 1000)

print(' training: {0}\n test: {1}\n'\
  .format(ratings_train.count(),
          ratings_test.count()
         ))


from pyspark.ml.recommendation import ALS
#https://spark.apache.org/docs/2.1.0/ml-collaborative-filtering.html
ratings_als = ALS(userCol = "userID",
                  itemCol = "artistID",
                  ratingCol = "play",
                  rank = 10, #latent components
                  maxIter = 10,
                  regParam = 0.1, #regularization
                  implicitPrefs = False, 
                  coldStartStrategy = "drop", 
                  )

 training: 64899
 test: 27935



Here we got a prediction for some 10 people

In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
ratings_model = ratings_als.fit(ratings_train)

ratings_pred = ratings_model.transform(ratings_test)
ratings_pred.show(10)

+------+--------+------+----+------------+----------+
|userID|artistID|weight|play|counartistID|prediction|
+------+--------+------+----+------------+----------+
|   463|    6397|    37|   1|          10| 0.9002598|
|    85|    2366|   120|   1|           7| 0.8998237|
|   126|    1580|   267|   1|          43|0.89969134|
|   210|     463|   510|   1|          17| 0.9002897|
|    12|     496|  3407|   1|          11|0.90015197|
|   336|    5803|   386|   1|           3|  0.897562|
|   476|    2122|   158|   1|           4|0.89997315|
|   348|     471|    60|   1|          42|0.90017974|
|    54|    1591|   112|   1|           3| 0.9004644|
|   419|    3794|     7|   1|           3| 0.8995972|
+------+--------+------+----+------------+----------+
only showing top 10 rows



Calculating mistake which give us some information about the goodness of the model

In [36]:
from pyspark.ml.evaluation import RegressionEvaluator
ratings_eval = RegressionEvaluator(metricName = "rmse", 
                                    labelCol = "play",
                                    predictionCol = "prediction")
ratings_rmse = ratings_eval.evaluate(ratings_pred)
print("RMSE = " + str(ratings_rmse))

RMSE = 0.10248847153410813


Recommendation for some users

In [37]:
user_recs=ratings_model.recommendForAllUsers(5).show(truncate=False)

+------+----------------------------------------------------------------------------------------------------+
|userID|recommendations                                                                                     |
+------+----------------------------------------------------------------------------------------------------+
|12    |[{4344, 0.9054272}, {8670, 0.9050417}, {2101, 0.90460163}, {1126, 0.90374666}, {1910, 0.90306634}]  |
|22    |[{103, 0.9075376}, {8670, 0.90601206}, {7148, 0.90586746}, {1888, 0.90547013}, {1923, 0.90533346}]  |
|26    |[{103, 0.90829647}, {1888, 0.9075561}, {1923, 0.9054516}, {2101, 0.90533996}, {8136, 0.9051457}]    |
|27    |[{4344, 0.90520436}, {2101, 0.90509385}, {8670, 0.90447515}, {1126, 0.90387815}, {1910, 0.9030821}] |
|28    |[{103, 0.9057022}, {2101, 0.90532345}, {1888, 0.9049002}, {4344, 0.9041218}, {1923, 0.9038287}]     |
|31    |[{2101, 0.9052906}, {4344, 0.9046452}, {103, 0.9043518}, {1888, 0.90395004}, {1126, 0.90382975}]    |
|34    |[{